# 🚀 S1W3D5 (Day 19)：模型保存与调优 (Integration & Tuning)

**今日任务**

1.  **逻辑实现**：编写“保存最佳模型 (Save Best Model)”的逻辑。
2.  **完整训练**：运行一次完整的 5-10 Epoch 训练，产出 `best_model.pth`。
3.  **理论深究**：Batch Size 到底怎么影响模型？

## 🛠️ 为什么不仅是 `torch.save`？

在 W2 我们学过 `torch.save(model.state_dict(), 'model.pth')`。但在实际项目中，我们不能训练完最后一步再保存。

**面试点：Early Stopping & Model Checkpointing**

  * **现象**：模型可能在第 3 个 Epoch 准确率最高 (98%)，到了第 10 个 Epoch 反而过拟合了 (Accuracy 降到 97%)。
  * **策略**：我们需要在这个 Loop 中加一个\*\*“裁判”\*\*：
      * 记录当前的 `best_accuracy`。
      * 每轮评估完，比较 `current_acc` 和 `best_accuracy`。
      * 如果**破纪录了**，立刻保存当前模型，并更新 `best_accuracy`。
      * 如果没破纪录，什么都不做。

## 💻 编写完整训练脚本 (Notebook)

请在你的新 Notebook 中运行以下代码。这段代码集成了我们之前封装的 `src`，并加入了**保存最佳模型**的核心逻辑。

In [ ]:
import sys
import os
import torch
import torch.optim as optim
import time

# 1. 挂载项目根目录
project_root = os.path.abspath(os.path.join(os.getcwd(), "../..")) 
if project_root not in sys.path:
    sys.path.append(project_root)

experiment_dir = os.path.join(project_root, 'experiments')
os.makedirs(experiment_dir, exist_ok=True)

# 2. 导入模块
from src.utils import get_logger, get_data_loaders
from src.models import SimpleCNN
from src.engine import train, evaluate

# 3. 准备工作
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 这里我们可以尝试调节 batch_size 看看效果
train_loader, test_loader = get_data_loaders(batch_size=16, data_root=os.path.join(project_root, 'data'))

# 4. 初始化 Logger (还是用它写文件，不用纠结屏幕显示了)
log_path = os.path.join(experiment_dir, 'final_training.log')
logger = get_logger(log_path)
# logger = get_logger('../../experiments/final_training.log')

model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- 5. 核心逻辑：保存最佳模型 ---
EPOCHS = 5  # 我们跑 5 轮试试
best_acc = 0.0 # 初始化最高分
save_path = 'best_model.pth' # 模型保存路径

logger.info(f"🚀 Start Final Training... Device: {device}, Batch Size: {train_loader.batch_size}")

start_time = time.time()

for epoch in range(1, EPOCHS + 1):
    # A. 训练
    train(model, device, train_loader, optimizer, epoch, log_interval=100, logger=logger)
    
    # B. 评估 (获取当前准确率)
    test_acc = evaluate(model, device, test_loader, logger=logger)
    
    # C. 裁判逻辑 (Save Best)
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), save_path)
        
        # 打印高光时刻
        msg = f"💾 Model Saved! (Accuracy improved: {best_acc:.2f}%)"
        print(msg) 
        logger.info(msg)
    else:
        print(f"⏳ No improvement (Best: {best_acc:.2f}%)")

total_time = time.time() - start_time
logger.info(f"✨ Training Finished in {total_time:.2f}s. Best Accuracy: {best_acc:.2f}%")
print(f"\n🏆 最终最佳准确率: {best_acc:.2f}%")

Train Epoch 1:   3%|▎         | 111/3750 [00:01<00:37, 98.26it/s, loss=0.2015]

Train Epoch: 1 [100/3750 (3%)]	Loss: 0.431391


Train Epoch 1:   6%|▌         | 214/3750 [00:02<00:35, 98.56it/s, loss=0.0984]

Train Epoch: 1 [200/3750 (5%)]	Loss: 0.179231


Train Epoch 1:   8%|▊         | 310/3750 [00:03<00:34, 98.69it/s, loss=0.3818] 

...

Train Epoch 1:  11%|█         | 418/3750 [00:04<00:33, 100.39it/s, loss=0.0995]

Train Epoch: 1 [400/3750 (11%)]	Loss: 0.039944


Train Epoch 1:  14%|█▎        | 511/3750 [00:05<00:32, 98.80it/s, loss=0.0432] 

Train Epoch: 1 [500/3750 (13%)]	Loss: 0.057517


KeyboardInterrupt: 

## 🧠 3. 理论补课：Batch Size 的玄学 (面试必问)

在等待模型训练的时候（5个 Epoch 应该很快），请认真阅读并思考这个问题。这是深度学习面试中**最经典**的超参数问题。

**Q: Batch Size (BS) 设得大一点好，还是小一点好？**

#### 🔴 大 Batch Size (例如 1024, 2048)

  * **优点**：
      * **快**：GPU 并行效率极高，训练一个 Epoch 时间短。
      * **稳**：梯度计算非常准确（取了 1000 张图的平均），Loss 曲线平滑，不震荡。
  * **缺点 (致命)**：
      * **泛化能力差 (Generalization Gap)**：这是 Yann LeCun 等大佬反复强调的。因为梯度太稳了，模型容易直接滑落到最近的一个\*\*“尖锐的极小值” (Sharp Minima)\*\* 坑里出不来。这种坑在训练集上 Loss 很低，但测试集稍微变一点数据，Loss 就飙升。
      * **内存溢出**：显存容易炸。

#### 🔵 小 Batch Size (例如 32, 64)

  * **优点**：
      * **泛化能力强**：这是反直觉的。因为样本少，每次计算的梯度都有点“不准”（自带噪声）。这种**噪声**其实是好事！它就像模型在下山时喝醉了，走路摇摇晃晃，反而更容易**跳出**那些尖锐的小坑，最终找到一个\*\*“平坦的极小值” (Flat Minima)\*\*。平坦的坑，泛化性最好。
  * **缺点**：
      * **慢**：GPU 等待时间多，训练时间长。
      * **震荡**：Loss 曲线像心电图一样跳动，很难收敛到极致的 0。

#### ⚖️ 黄金法则 (Rule of Thumb)

1.  **默认起手式**：`32` 或 `64`。这是最安全的范围。
2.  **线性缩放法则 (Linear Scaling Rule)**：如果你非要增加 Batch Size 来加速（比如加到 256），你通常需要**同比例增加学习率 (Learning Rate)**。
      * $BatchSize \times k \rightarrow LearningRate \times k$。


## 🔬 每日实验 (你的任务)

1.  **运行代码**：确保上面的脚本跑通，并看到 `💾 Model Saved!` 的提示。
2.  **检查产出**：
      * 看看你的目录下是不是多了一个 `best_model.pth` 文件？
      * 文件大小是多少？（应该在几百 KB 到 1 MB 之间，很小）。
3.  **简单调参 (选做)**：
      * 将 `batch_size` 改为 **16**，再跑一次。观察：
          * 训练时间变长了吗？
          * Loss 下降得是更平滑了还是更抖动了？

完成这步，我们的**训练阶段**就彻底毕业了！你可以自豪地把 `best_model.pth` 也就是你的“智障 AI”（现在已经不智障了）存起来了。

明天 **W3D6**，我们将学习如何写一个 `predict.py` 脚本，加载这个 `.pth` 文件，去识别一张它**从未见过**的、甚至是你自己手写的数字图片！

运行完告诉我结果！